# Coleta de Dados no Twitter utilizando pacote Tweepy do Python e armazena em arquivo, ingestão no SQL Server

In [ ]:
# Install package
! pip install tweepy

In [1]:
import json

In [ ]:
# Run script to get tweets
%run get_tweets.py


In [2]:
# Open the file
with open("collected_tweets_2020-11-25-15-40-41.txt", "r") as file:
    tweets = file.readlines()
    

In [ ]:
# Write a JSON example
with open ("tweet.json", "w") as file2:
    print(json.dump(
        json.loads(
            json.loads(tweets[0]
                      )
        ), file2))


In [3]:
# List with tweets parsed in JSON format
parsed_tweets = [json.loads( json.loads(tweet) ) for tweet in tweets]


In [4]:
# Checking the keys
parsed_tweets[0].keys()


dict_keys(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'extended_tweet', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])

## Tranformando os Tweets em uma estrutura relacional Pandas Dataframe

#### Define Functions

In [5]:
# Import Library
import pandas as pd

def create_df_and_drop_columns(parsed_tweet):
    try:
        df_treated = pd.DataFrame(parsed_tweet).reset_index(drop=True).iloc[:1]

        df_treated.drop(columns=['quote_count', 'reply_count', 'retweet_count',
                             'favorite_count', 'favorited', 'retweeted', 
                             'user', 'retweeted_status', 'entities'], inplace=True)

        return df_treated
    
    except:
        return None

# ------------------------------------------------------------------------------

def tweet_to_df_(parsed_tweet):
    try:
        user_mentions = [user for user in parsed_tweet["entities"]["user_mentions"]]

        df = pd.DataFrame(user_mentions).rename(columns={
                                        "screen_name": "entities_screen_name",
                                        "name": "entities_name",
                                        "id": "entities_id",
                                        "id_str": "entities_id_str"
                                    })

        return df
    except:
        return None
    
# ------------------------------------------------------------------------------
    

def rename_columns(df, parsed_tweet):
    df["user_id"] = parsed_tweet["user"]["id"]
    df["user_id_str"] = parsed_tweet["user"]["id_str"]
    df["user_screen_name"] = parsed_tweet["user"]["screen_name"]
    df["user_location"] = parsed_tweet["user"]["location"]
    df["user_description"] = parsed_tweet["user"]["description"]
    df["user_protected"] = parsed_tweet["user"]["protected"]
    df["user_verified"] = parsed_tweet["user"]["verified"]
    df["user_followers_count"] = parsed_tweet["user"]["followers_count"]
    df["user_friends_count"] = parsed_tweet["user"]["friends_count"]
    df["user_created_at"] = parsed_tweet["user"]["created_at"]
    
    return df


In [6]:
# list of Pandas Dataframe object
treateds_df = [create_df_and_drop_columns(tweet) for tweet in parsed_tweets]

In [7]:
# Remove None values
treateds_df = [df for df in treateds_df if df is not None]

In [8]:
# Verify len
len(treateds_df)

1061

In [9]:
# Concat all Dataframes in treateds_df list
final_df = pd.concat(treateds_df, ignore_index=True)

In [10]:
# check final df
final_df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,is_quote_status,filter_level,lang,timestamp_ms,possibly_sensitive,extended_entities
0,Wed Nov 25 18:40:18 +0000 2020,1331669054852100099,1331669054852100099,RT @IvyKungu: why are you saying this like it’...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,1.331330e+18,1331330446458638336,1331330446458638336,NaN,True,low,en,1606329618548,NaN,NaN
1,Wed Nov 25 18:40:18 +0000 2020,1331669054977961984,1331669054977961984,"RT @EinsteinsMagic: Racist Demonic Obama, blam...","<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,False,low,en,1606329618578,NaN,NaN
2,Wed Nov 25 18:40:18 +0000 2020,1331669054956953600,1331669054956953600,RT @gatewaypundit: We spoke with a top Trump a...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,False,low,en,1606329618573,NaN,NaN
3,Wed Nov 25 18:40:18 +0000 2020,1331669054923411457,1331669054923411457,RT @donwinslow: And tens of millions of us are...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,...,1.331654e+18,1331653915553751042,1331653915553751042,NaN,True,low,en,1606329618565,False,NaN
4,Wed Nov 25 18:40:18 +0000 2020,1331669055137263619,1331669055137263619,"RT @jimsciutto: “My family voted for him, dona...","<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,False,low,en,1606329618616,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,Wed Nov 25 18:40:51 +0000 2020,1331669192827936769,1331669192827936769,RT @marceelias: 🚨NEW: Republicans have dismiss...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,False,low,en,1606329651444,NaN,NaN
1057,Wed Nov 25 18:40:51 +0000 2020,1331669194459537408,1331669194459537408,RT @EpochTimes: “We now believe that Donald Tr...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,False,low,en,1606329651833,NaN,NaN
1058,Wed Nov 25 18:40:51 +0000 2020,1331669194455326724,1331669194455326724,RT @realDonaldTrump: https://t.co/e7giu2jEG1,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,1.331329e+18,1331328530391719937,1331328530391719937,NaN,True,low,und,1606329651832,False,NaN
1059,Wed Nov 25 18:40:51 +0000 2020,1331669194652278784,1331669194652278784,RT @dbongino: Only a Democrat could say someth...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,NaN,NaN,NaN,NaN,False,low,en,1606329651879,False,NaN


### Ingestão de dados do Twitter no SQL Server

In [11]:
# import libraries
import pyodbc
import sqlalchemy

In [12]:
# create engine connection with sql server
engine = sqlalchemy.create_engine(
    "mssql+pyodbc://[server_name]/twitter?driver=ODBC+Driver+17+for+SQL+Server?Trusted_Connection=yes"
)

In [ ]:
# inserting tweets (Pandas Dataframe) into SQL server Database
final_df.to_sql("tweets", con=engine, index=False, if_exists='append')